In [72]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

model = read_sbml_model('Models/iML1515.xml')

# Print all the reactions involving extra-cellular L-aspartate
for reaction in model.metabolites.asp__L_e.reactions:
    print(reaction.id, reaction, reaction.name)


# Set the objective function for the biomass model
model.objective= 'BIOMASS_Ec_iML1515_core_75p37M' 
model_ojective_value = print (model.optimize().objective_value)


#set the optimal value for biomass to 100%
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.upper_bound=0.873920
#set the lower bound to not be less than 75 % of the optimal value
model.reactions.BIOMASS_Ec_iML1515_core_75p37M.lower_bound=0.873920*0.75
#print the new objective value
model_BM_optimized = print(model.optimize().objective_value)


# Setting a new objective function based on L-asp production rate
with model:
    model.objective = model.reactions.EX_asp__L_e
    model_asp_optimized = print (model.optimize().objective_value)



EX_asp__L_e EX_asp__L_e: asp__L_e -->  L-Aspartate exchange
ASPtex ASPtex: asp__L_e <=> asp__L_p L-aspartate transport via diffusion (extracellular to periplasm)
0.87699721425716
0.87392
4.596027557488181


In [73]:
from cobra.flux_analysis.phenotype_phase_plane import production_envelope


In [74]:
from re import template
import plotly.express as px

medium = model.medium
with model:
    model.medium = medium
    model.objective = model.reactions.EX_asp__L_e
    asp_production = model.optimize().objective_value
    ppp_BIOMASS = production_envelope(model, reactions=[model.reactions.EX_asp__L_e], objective=model.reactions.BIOMASS_Ec_iML1515_core_75p37M)
    ppp_asp = production_envelope(model, reactions=[model.reactions.EX_glc__D_e], objective=model.reactions.EX_asp__L_e)
    ppp_o2 = production_envelope(model, reactions=[model.reactions.EX_o2_e], objective=model.reactions.EX_asp__L_e)

    fig1 = px.line(ppp_BIOMASS, x='EX_asp__L_e', y='flux_maximum', title='Biomass vs. L-aspartate production', template="ggplot2")
    fig1.update_layout(height=600, width=800, title_x=0.5)  # Update figure size
    fig1.update_traces(line_color='blue')  # Update line color
    fig1.update_xaxes(title_text='L-aspartate[mmol/g DW/h]')  # Set x-axis label
    fig1.update_yaxes(title_text='Biomass objective value[1/h]')  # Set y-axis label

    fig2 = px.line(ppp_asp, x='EX_glc__D_e', y='flux_maximum', title='L-asp production vs. glucose consumption', template="ggplot2")
    fig2.update_layout(height=600, width=800, title_x=0.5)  # Update figure size
    fig2.update_traces(line_color='green')  # Update line color
    fig2.update_xaxes(title_text='Glucose uptake[mmol/g DW/h]')  # Set x-axis label
    fig2.update_yaxes(title_text='L-aspartate objective value[mmol/g DW/h]')  # Set y-axis label

    fig3 = px.line(ppp_o2, x='EX_o2_e', y='flux_maximum', title='L-asp production vs. oxygen consumption', template="ggplot2")
    fig3.update_layout(height=600, width=800, title_x=0.5)  # Update figure size
    fig3.update_traces(line_color='red')  # Update line color
    fig3.update_xaxes(title_text='Oxygen consumption[mmol/g DW/h]')  # Set x-axis label
    fig3.update_yaxes(title_text='L-aspartate objective value[mmol/g DW/h]')  # Set y-axis label

    fig1.show()
    fig2.show()
    fig3.show()


## Let's look what happens when we change the sugar source!

In [75]:
medium = model.medium
with model:
    medium['EX_glc__D_e'] = 0
    medium['EX_succ_e'] = 10
    model.medium = medium
    solution = model.optimize()
    print(solution.fluxes['EX_asp__L_e'])

   

-4.689887559800871


C:\Users\criss\AppData\Roaming\Python\Python310\site-packages\cobra\util\solver.py:554: UserWarning:

Solver status is 'infeasible'.



In [76]:
model.medium


{'EX_pi_e': 1000.0,
 'EX_co2_e': 1000.0,
 'EX_fe3_e': 1000.0,
 'EX_h_e': 1000.0,
 'EX_mn2_e': 1000.0,
 'EX_fe2_e': 1000.0,
 'EX_glc__D_e': 10.0,
 'EX_zn2_e': 1000.0,
 'EX_mg2_e': 1000.0,
 'EX_ca2_e': 1000.0,
 'EX_ni2_e': 1000.0,
 'EX_cu2_e': 1000.0,
 'EX_sel_e': 1000.0,
 'EX_cobalt2_e': 1000.0,
 'EX_h2o_e': 1000.0,
 'EX_mobd_e': 1000.0,
 'EX_so4_e': 1000.0,
 'EX_nh4_e': 1000.0,
 'EX_k_e': 1000.0,
 'EX_na1_e': 1000.0,
 'EX_cl_e': 1000.0,
 'EX_o2_e': 1000.0,
 'EX_tungs_e': 1000.0,
 'EX_slnt_e': 1000.0}